# 11-дәріс: Ағаштар ансамбльдері. Кездейсоқ Орман

**Дәріс мақсаты:**
1.  Модельдердің сапасы мен тұрақтылығын арттыру тәсілі ретінде **ансамбльдік әдістер** тұжырымдамасымен танысу.
2.  Кездейсоқ орманның негізі ретінде **бэггинг (Bagging)** әдісін талдау.
3.  **Кездейсоқ Орман (Random Forest)** алгоритмін, оның артықшылықтарын және практикалық қолданылуын зерттеу.
4.  Балама тәсіл — **бустингке (Boosting)** қысқаша тоқталу.

### 1-бөлім: Неліктен бір ағаш жеткіліксіз? Көпшілік даналығы

Өткен дәрісте біз шешімдер ағаштарының елеулі кемшілігі бар екенін анықтадық: олар **тұрақсыз** және **артық оқытуға** қатты бейім. Деректердегі шамалы өзгеріс ағаштың құрылымын түбегейлі өзгертуі мүмкін.

**Бұл мәселені қалай шешуге болады?** Күнделікті өмірдегі даналыққа жүгінейік. Маңызды шешім қабылдағымыз келгенде, біз сирек бір сарапшының пікіріне сүйенеміз. Біз бірнеше пікір жинап, қорытынды шешімді консенсус негізінде қабылдаймыз. Бұл принцип **"көпшілік даналығы"** деп аталады.

Машиналық оқытуда бұл тәсіл **ансамбльдеу** деп аталады. Идеясы — көптеген әртүрлі (әлсіз немесе тұрақсыз) модельдерді оқытып, олардың болжамдарын біріктіріп, бір, бірақ дәлірек және робасты болжам алу.

![ensamble](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/ensamble.png)

### 2-бөлім: Бэггинг (Bagging = Bootstrap Aggregating)

Бэггинг — ең танымал ансамбльдік әдістердің бірі және кездейсоқ орманның негізі. Оның атауы екі бөліктен тұрады.

#### 1-қадам: Bootstrap (Бутстрэп)

Біздің модельдеріміз (ағаштар) әртүрлі болуы үшін, оларды **әртүрлі деректерде** оқытуымыз керек. Бірақ бізде тек бір ғана оқыту таңдамасы бар. Басқаларын қайдан аламыз?

Бутстрэп — бұл `N` өлшемді бір таңдамадан сол `N` өлшемді көптеген **ішкі таңдамалар** генерациялауға мүмкіндік беретін статистикалық әдіс. Бұл **бастапқы таңдамадан объектілерді қайтару арқылы кездейсоқ таңдау** жолымен жасалады.

![bootstrap](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/bootstrap1.png)
Тағы бір мысал
![bootstrap1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/bootstrapping.png)

Нәтижесінде әрбір ішкі таңдама бірегей болады: кейбір бастапқы объектілер онда қайталанады, ал кейбіреулері (орташа есеппен 37%-ға жуығы) мүлдем түспейді.

#### 2-қадам: Aggregation (Агрегаттау)

Алынған `T` бутстрэп-ішкі таңдамалардың әрқайсысында біз **өз тәуелсіз шешімдер ағашын** $b_t(x)$ оқытамыз. Әдетте бұл ағаштарды өте терең етіп жасайды, оларға өз деректер бөлігінде әдейі артық оқытуға мүмкіндік береді.

Жаңа объект үшін болжам жасау қажет болғанда, біз барлық `T` болжамды жинап, оларды агрегаттаймыз.

**Классификация үшін — мажоритарлық дауыс беру:**
Қорытынды болжам $a(x)$ ретінде ағаштардың көпшілігі дауыс берген класс алынады.
$$ a(x) = \text{sign} \left( \frac{1}{T} \sum_{t=1}^{T} b_t(x) \right) $$

**Регрессия үшін — орташалау:**
Қорытынды болжам $a(x)$ ретінде барлық ағаштардың болжамдарының арифметикалық ортасы алынады.
$$ a(x) = \frac{1}{T} \sum_{t=1}^{T} b_t(x) $$

Нәтижесінде жекелеген артық оқытылған ағаштардың қателері мен "қызықтары" бір-бірін өтейді, және ансамбльдің қорытынды болжамы әлдеқайда дәлірек және тұрақты болады.

#### Математикалық негіздеме: орташалау қатені қалай азайтады

Неліктен "көпшілік даналығы" жұмыс істейді? Бұл әсерді ықтималдықтар теориясының негіздері арқылы оңай түсіндіруге болады.

Регрессия есебін шешіп жатырмыз делік. Айталық:
*   **$\theta$** — біз болжағымыз келетін шын, дұрыс мән.
*   **$x_j$** — $j$-ші "сарапшының" (немесе біздің ансамбльдегі $j$-ші ағаштың) жауабы.

Әрбір жауапты аддитивті модель түрінде көрсетуге болады:
$$ x_j = \theta + \eta_j $$
мұндағы $\eta_j$ — $j$-ші сарапшының кездейсоқ қатесі.

Қарапайымдылық үшін екі қисынды болжам жасаймыз:
1.  **Қателер ығыспаған:** Орташа есеппен, сарапшылар жүйелі түрде бір жаққа қателеспейді. Математикалық тұрғыдан: қатенің орташа мәні нөлге тең, $E[\eta_j] = 0$.
2.  **Қателер тәуелсіз:** Бір сарапшының қатесі екіншісінің қатесімен байланысты емес. Математикалық тұрғыдан: $i \neq j$ үшін $E[\eta_i \eta_j] = 0$. Бұл — біз бутстрэп пен `max_features` қолданатын негізгі шарт!

Әрбір жеке сарапшы қатесінің **дисперсиясы** $\sigma^2$ болсын. Дисперсия — бұл болжамдардың "шашырау" немесе тұрақсыздық өлшемі.
$$ \sigma^2 = E[\eta_j^2] $$

Енді $T$ сарапшының жауаптарын орташалау арқылы ансамбльіміздің болжамын табайық:
$$ y = \frac{1}{T} \sum_{j=1}^{T} x_j $$

Осы орташаланған жауаптың қатесінің дисперсиясы қандай болады?
$$ E[(\theta - y)^2] = E\left[\left(\theta - \frac{1}{T} \sum_{j=1}^{T} x_j\right)^2\right] = E\left[\left(\frac{1}{T} \sum_{j=1}^{T} (\theta - x_j)\right)^2\right] = E\left[\left(\frac{1}{T} \sum_{j=1}^{T} \eta_j\right)^2\right] $$

Қосындының квадратын ашып, қателердің тәуелсіздігі қасиетін пайдалансақ, мынаны аламыз:
$$ \frac{1}{T^2} \sum_{j=1}^{T} E[\eta_j^2] = \frac{1}{T^2} \cdot T \cdot \sigma^2 = \frac{\sigma^2}{T} $$

**Қорытынды:**
> Орташаланған жауаптың (ансамбльдің) дисперсиясы жеке бір ағаштың жауабының дисперсиясынан **$T$ есе аз**.

**Қарапайым сөзбен айтқанда:** $T$ тәуелсіз, бірақ шулы модельдердің болжамдарын орташалау арқылы біз қорытынды "шуды" (қатені) $T$ есе азайтамыз. Дәл осы себепті бэггинг пен кездейсоқ орман жалғыз ағаштардың тұрақсыздығымен және артық оқытылуымен тиімді күреседі. Біз қаншалықты көп ағаш қоссақ, олардың жеке қателері соншалықты қатты тегістеледі.

### 3-бөлім: Кездейсоқ Орман (Random Forest) — "ерекшелігі" бар Бэггинг

**Кездейсоқ орман** — бұл ағаштар бэггингінің жетілдірілген нұсқасы. Ол ағаштарды бір-бірінен одан да тәуелсіз ету үшін тағы бір кездейсоқтық элементін қосады.

**Бэггингтен айырмашылығы:**
Әрбір ағаштағы **әрбір түйінді** құру кезінде алгоритм ең жақсы сплитті **барлық** қолжетімді белгілерден емес, тек сол белгілердің **кездейсоқ ішкі жиынынан** таңдайды.

Бұл параметр `scikit-learn`-де `max_features` деп аталады.

**Бұл не үшін қажет?**
Деректеріңізде бір өте күшті, басым белгі бар деп елестетіңіз. Кәдімгі бэггингте көптеген ағаштар түбірдегі бірінші бөлу үшін дәл сол белгіні таңдайды. Нәтижесінде барлық ағаштар бір-біріне өте ұқсас болады ("корреляцияланған"), және орташалаудан болатын әсер әлсірейді.

Кездейсоқ орман кейбір ағаштарды осы күшті белгіге **қол жеткізбестен** құруға мәжбүрлейді, оларды деректердегі басқа, баламалы заңдылықтарды табуға итермелейді. Бұл ансамбльдегі "пікірлердің әртүрлілігін" арттырады және әдетте қорытынды дәлдікті жоғарылатады.

![randomforest1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/random_forest1.png)
![randomforest1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/randomforest1.png)
![randomforest1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/randomforest2.png)

### 4-бөлім: Тарихи анықтама және Scikit-Learn-дегі іске асырылуы

#### Негізгі тұлғалар мен идеялар
*   **CART (Classification and Regression Trees):** `scikit-learn`-дегі жеке ағаштарды құру негізінде жатқан алгоритмді **Лео Брейман** және т.б. 1984 жылы ұсынған.
*   **Random Subspace Method:** Классификаторларды белгілердің кездейсоқ ішкі жиындарында оқыту идеясын **Тин Кам Хо** 1998 жылы ұсынған. Бұл `max_features` параметрінің негізі.
*   **Бэггинг және Кездейсоқ Орман:** **Лео Брейман** 1994 жылы бэггингті ұсынды, ал 2001 жылы оны кездейсоқ ішкі кеңістіктер әдісімен біріктіріп, **Random Forest** алгоритмін ресімдеп, танымал етті.

#### Scikit-Learn-дегі іске асырылуы
`scikit-learn`-де екі негізгі класс бар:
- `sklearn.ensemble.RandomForestClassifier` — классификация есептері үшін.
- `sklearn.ensemble.RandomForestRegressor` — регрессия есептері үшін.

**Негізгі гиперпараметрлер:**
- `n_estimators`: Ормандағы ағаштар саны. Ең маңызды параметр. Неғұрлым көп болса, соғұрлым жақсы (белгілі бір шекке дейін), бірақ оқыту ұзағырақ болады.
- `max_features`: Ең жақсы сплитті іздеу кезінде қарастырылатын белгілер саны. Әдеттегі мәндер: `'sqrt'` (жалпы белгілер санының квадрат түбірі), `'log2'` немесе нақты сан.
- `criterion`: Сплит сапасын бағалау критерийі (`'gini'` немесе `'entropy'` классификация үшін, `'squared_error'` регрессия үшін).
- `max_depth`, `min_samples_split`, `min_samples_leaf`: Әрбір жеке ағашты реттеуге арналған параметрлер (`DecisionTreeClassifier`-ге ұқсас).
- `bootstrap`: `True` (әдепкі) немесе `False`. Бутстрэп механизмін қосады немесе өшіреді.
- `oob_score`: `True` немесе `False`. Out-of-Bag бағалауын қосады немесе өшіреді.

#### Out-of-Bag (OOB) Score дегеніміз не?

Бұл Кездейсоқ Орманның ең талғампаз ерекшеліктерінің бірі.

**Идея:** №1 ағашты оқыту үшін бутстрэп-ішкі таңдамасын жасағанда, бастапқы деректердің шамамен 37%-ы оған түспеді. Осы "қалдырылған" деректерді дәл осы №1 ағаш үшін **валидациялық жиын** ретінде пайдалануға болады.

**Процесс:**
1.  Бастапқы оқыту таңдамасындағы **әрбір** $x_i$ объектісі үшін біз осы объект **пайдаланылмаған** (яғни "out-of-bag" болған) барлық ағаштарды табамыз.
2.  Біз сол ағаштардан $x_i$ объектісі үшін болжам жасауды сұраймыз.
3.  $x_i$ объектісі үшін қорытынды "OOB-болжамын" алу үшін олардың болжамдарын орташалаймыз.
4.  Барлық OOB-болжамдарды нақты $y_i$ белгілерімен салыстырып, **Out-of-Bag Score** аламыз — бұл бөлек валидациялық жиын құруды қажет етпестен алынған модель сапасының шынайы бағасы!

Бұл мүмкіндікті пайдалану үшін `bootstrap=True` (бұл әдепкі мән) және `oob_score=True` деп орнату керек.

![ensamble](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/randomforest4.png)

#### Тереңірек үңілу: `max_features` не үшін қажет және оның мәндері қайдан шыққан?

`max_features` параметрі — Кездейсоқ Орманды жай ғана ағаштар Бэггингінен ерекшелейтін "айрықша белгі". Оның не үшін қажет екенін және неліктен оған $\sqrt{n_{features}}$ сияқты таңқаларлық мәндер қолданылатынын қарастырайық.

**Басты мақсат: Ағаштарды декорреляциялау**

Есімізде болса, "көпшілік даналығы" көпшіліктің пікірлері **тәуелсіз** және **әртүрлі** болғанда ең жақсы жұмыс істейді. Егер барлық "сарапшылар" (ағаштар) бірдей ойласа, онда ансамбль бір сарапшыдан ақылды болмайды.

**Мәселе:** Деректеріңізде бір өте күшті, басым белгі бар деп елестетіңіз (мысалы, қымбат тауарды сатып алуды болжау кезінде "табыс"). Егер біз ағаштарды қарапайым бэггинг әдісімен құратын болсақ, онда әрбір ағаш дерлік ең бірінші, түбірлік бөлу үшін дәл осы белгіні таңдауы мүмкін. Нәтижесінде ормандағы барлық ағаштар бір-біріне өте ұқсас болады — олар **корреляцияланады**. Бұдан ансамбльдің тиімділігі қатты төмендейді.

**Шешім:** `max_features` параметрі бұл мәселені әрбір түйін үшін "таңдауды" жасанды түрде шектеу арқылы шешеді. Кезекті ағаштағы кезекті түйінді құру кезінде алгоритм мынаны орындайды:
1.  Деректер жиынтығындағы **барлық** қолжетімді белгілерді алады.
2.  Олардан `max_features` өлшеміндегі **кездейсоқ ішкі жиынды** таңдайды.
3.  Ең жақсы сплитті **тек осы ішкі жиынның белгілері арасынан** іздейді.

Бұл кейбір ағаштарды басым белгіні "көрмей-ақ" құрылуға мәжбүрлейді және оларды деректердегі басқа, айқын емес заңдылықтарды табуға итермелейді. Бұл — **декорреляция**.

**Негізгі ымыра (Trade-off)**

`max_features` таңдауы — бұл жеке ағаштардың күші мен олардың әртүрлілігі арасындағы тепе-теңдік:

*   **`max_features`-тің кіші мәні:**
    *   **Артықшылығы:** Ағаштар өте әртүрлі болып шығады (төмен корреляция). Бұл "көпшілік даналығы" үшін жақсы.
    *   **Кемшілігі:** Жеке ағаштар өте "әлсіз" болуы мүмкін, себебі олардың маңызды белгілерге қол жеткізуі болмауы мүмкін. Олардың жеке болжамдық күші төмендейді.

*   **`max_features`-тің үлкен мәні:**
    *   **Артықшылығы:** Әрбір ағаш күшті болады, себебі оның барлық дерлік белгілерге қол жеткізуі бар.
    *   **Кемшілігі:** Ағаштар бір-біріне өте ұқсас болып кетеді (жоғары корреляция), және ансамбль өз тиімділігін жоғалтады. Егер `max_features` жалпы белгілер санына тең болса, онда біз кәдімгі Бэггинг аламыз.

**$\sqrt{n}$ және $\log_2(n)$ мәндері қайдан алынған?**

Бұл мәндер қатаң математикалық теореманың нәтижесі емес, **Лео Брейман** өзінің Кездейсоқ Орман туралы түпнұсқа жұмысында ұсынған **эвристикалар** (тәжірибелік ережелер) болып табылады. Олар әртүрлі деректер жиындарында көптеген эксперименттер барысында эмпирикалық жолмен табылған және өздерін тамаша бастапқы нүкте ретінде көрсеткен.

*   `max_features='sqrt'` (немесе $\sqrt{n_{features}}$):
    *   **Неліктен бұл жақсы?** Бұл жақсы ымыра. Таңдалатын белгілердің саны жалпы белгілер санымен бірге өседі, бірақ баяу өседі. Бұл ағаштардың әртүрлілігін сақтауға мүмкіндік береді, бірақ сонымен бірге оларға маңызды предикторларды "көруге" жеткілікті жақсы мүмкіндік береді.
    *   **Қашан қолданылады?** Бұл `scikit-learn`-де **классификация есептері** үшін әдепкі мән болып табылады, және көп жағдайда оны өзгерту қажет емес.

*   `max_features='log2'` (немесе $\log_2(n_{features})$):
    *   **Неліктен бұл жақсы?** Бұл одан да қатаң шектеу. Таңдалатын белгілердің саны өте баяу өседі.
    *   **Қашан қолданылады?** Бұл нұсқа сізде **өте көп белгілер** (мыңдаған) болғанда және олардың көбі не ақпаратсыз (шу), не бір-бірімен қатты корреляцияланған деп күдіктенгенде пайдалы болуы мүмкін.

*   `max_features=1.0` (немесе `n_features`):
    *   **Бұл не?** Бұл кәдімгі Бэггингке тең. Алгоритм әр қадамда барлық белгілерді көреді.
    *   **Қашан қолданылады?** `scikit-learn`-де **регрессия есептері** үшін әдепкі бойынша қолданылады. Регрессия үшін күштірек жеке ағаштардың болуы маңыздырақ деп саналады, және корреляция мәселесі соншалықты өткір емес.

### 6-бөлім: Бустинг (Boosting) туралы қысқаша

Бустинг — бұл бэггингтен түбегейлі ерекшеленетін ансамбльдеудің тағы бір қуатты тәсілі.

| Бэггинг (Random Forest) | Бустинг (Gradient Boosting, AdaBoost) |
| :--- | :--- |
| Ағаштар **тәуелсіз** және **параллель** құрылады. | Ағаштар **бірізді** құрылады. |
| Әрбір ағаш басқалары туралы "білмейді". | Әрбір келесі ағаш алдыңғысының **қателерінен үйренеді**. |
| Мақсат — орташалау арқылы **дисперсияны (variance)** азайту. | Мақсат — қателерді түзету арқылы **ығысуды (bias)** азайту. |
| Терең, артық оқытылған ағаштар қолданылады. | Өте қарапайым, "әлсіз" ағаштар (жиі тереңдігі 1-3 болатын "түбіртектер") қолданылады. |

#### Бустинг интуициясы: Қателермен жұмыс

**Интуиция:** Күрделі есепті шешу үшін сарапшылар тобын жинап жатырсыз делік. Оларды параллель жұмыс істеуге сұраудың орнына, сіз мұны бірізді түрде жасайсыз:
1.  Бірінші сарапшы өз әрекетін жасайды.
2.  Сіз оның қателеріне қарап, екінші сарапшыдан біріншісі сәтсіздікке ұшыраған жағдайларға назар аударуды сұрайсыз.
3.  Үшінші сарапшы алғашқы екеуінің жұмысынан кейін қалған қателерге шоғырланады.
4.  Осылайша жалғаса береді. Қорытынды шешім — барлық сарапшылардың салмақталған "пікірі".

#### Мысал: AdaBoost (Adaptive Boosting) алгоритмі

AdaBoost — бустингтің алғашқы және ең көрнекі алгоритмдерінің бірі.

1.  **Бастапқы орнату:** Ең басында оқыту таңдамасының барлық объектілері бірдей **салмаққа** ие болады: $w_i = 1/N$.

2.  **Итеративті құру:** Әрбір $t=1, \dots, T$ қадамы үшін:
    a. Ағымдағы $w_i$ салмақтарын ескере отырып, деректерде "әлсіз" модель (мысалы, ағаш-түбіртек) $b_t(x)$ оқытылады. Модель үлкен салмағы бар объектілерді дұрыс жіктеуге тырысады.
    б. Осы модельдің $\epsilon_t$ қатесі есептеледі.
    в. Осы модельдің **"дауыс салмағы"** $\alpha_t$ есептеледі. Модельдің қатесі неғұрлым аз болса, оның соңғы ансамбльдегі дауысы соғұрлым "қатты" болады.
       $$ \alpha_t = \frac{1}{2} \ln{\frac{1 - \epsilon_t}{\epsilon_t}} $$
    г. **Объектілердің салмақтары жаңартылады:**
       *   **Дұрыс емес** жіктелген объектілердің салмақтары **артады**.
       *   **Дұрыс** жіктелген объектілердің салмақтары **азаяды**.
       Бұл келесі $b_{t+1}(x)$ моделін ең "күрделі" объектілерге назар аударуға мәжбүрлейді.

3.  **Соңғы болжам:** Қорытынды модель — барлық әлсіз модельдердің салмақталған дауыс беруі.

    $$ a(x) = \text{sign} \left( \sum_{t=1}^{T} \alpha_{t} b_{t}(x) \right) $$

**Gradient Boosting** бұл идеяны дамытады: әрбір келесі модель жай ғана қателерді емес, алдыңғы модельдің **шығын функциясының градиентін** болжауға үйренеді. Бұл жалпырақ және әдетте қуаттырақ тәсіл.

Бұл өте қуатты, бірақ баптау мен түсінуге күрделірек әдістер. Танымал іске асырулар — Gradient Boosting, XGBoost, LightGBM. Біздің курс аясында біз оларды егжей-тегжейлі қарастырмаймыз, бірақ бұл әрі қарай зерттеу үшін маңызды бағыт.

![baggingboosting](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/baggingboosting.png)

### 4.1-бөлім: Кездейсоқ Орманның "тегін түскі асы": неге ол ағаштар санының өсуімен артық оқытылмайды?

Бір шешімдер ағашының басты мәселесін еске түсірейік: ағаш неғұрлым терең (күрделі) болса, артық оқытылу қаупі соғұрлым жоғары. Интуитивті түрде ансамбльге барған сайын күрделі, артық оқытылған ағаштарды қосу арқылы біз қорытынды модельді одан да артық оқытылған етуіміз керек сияқты көрінеді.

Бірақ Кездейсоқ Орманмен таңғажайып нәрсе болады: **ағаштар санының (`n_estimators`) артуымен тесттік таңдамадағы қате (жалпылау қабілеті) өсуін тоқтатып, платоға шығады.**

**Бұл неліктен орын алады?**

Бұл әсер **Үлкен сандар заңының** салдары болып табылады. Ормандағы әрбір ағаш — бұл, шын мәнінде, өз болжамын жасайтын кездейсоқ шама. Оның жоғары ығысуы (variance) бар, яғни ол өзінің бутстрэп-таңдамасына қатты бейімделген.

Біз көптеген **тәуелсіз (немесе әлсіз корреляцияланған)** кездейсоқ шамалардың болжамдарын орташалағанда, олардың жеке қателері (дисперсия) бір-бірін жоя бастайды.

*   **Алғашқы ағаштар (1-ден ~20-ға дейін):** Әрбір жаңа ағаш көптеген жаңа, бірегей "ақпарат" әкеледі. Тесттік таңдамадағы қате күрт төмендейді.
*   **Келесі ағаштар (~20-дан ~100-200-ге дейін):** Жаңа ағаштар әлі де өз үлесін қосып, нәтижені жақсартады, бірақ әсері соншалықты күшті емес. Қате біртіндеп төмендей береді.
*   **Ағаштар санын одан әрі арттыру:** Ансамбль жеткілікті "дана" болады. Әрбір жаңа ағаш енді принципті түрде жаңа ақпарат қоспайды, тек жалпы көріністі аздап "нақтылайды". Тесттік таңдамадағы қате тұрақталып, **асимптотикалық платоға** шығады. Ол енді нашарламайды!

**Қорытынды:** Күрделілікпен ымыраға келуді қажет ететін көптеген басқа модельдерден айырмашылығы, Кездейсоқ Ормандағы `n_estimators` үшін ереже қарапайым: **"неғұрлым көп болса, соғұрлым жақсы"** (немесе, кем дегенде, жаман емес). Жалғыз шектеу — оқыту уақыты.

Мұны тәжірибеде көрейік.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Күрделірек деректер жиынтығын генерациялау
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=5,
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Тексеруге арналған ағаштар санының диапазоны
n_estimators_range = range(1, 201)

# Қателерді сақтауға арналған тізімдер
train_errors = []
test_errors = []

for n_estimators in n_estimators_range:
    # Модельді ағымдағы ағаштар санымен оқыту
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Болжамдар жасау
    train_preds = rf.predict(X_train)
    test_preds = rf.predict(X_test)
    
    # Қатені есептеу (1 - accuracy) және тізімдерге қосу
    train_errors.append(1 - accuracy_score(y_train, train_preds))
    test_errors.append(1 - accuracy_score(y_test, test_preds))

# График салу
plt.figure(figsize=(12, 7))
plt.plot(n_estimators_range, train_errors, label="Оқыту таңдамасындағы қате (Train Error)")
plt.plot(n_estimators_range, test_errors, label="Тесттік таңдамадағы қате (Test Error)", linewidth=2)

plt.ylabel("Қате деңгейі (1 - Accuracy)")
plt.xlabel("Ағаштар саны (n_estimators)")
plt.title("Кездейсоқ Ормандағы ағаштар санына қателіктің тәуелділігі", fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

### 5-бөлім: Кездейсоқ Орман іс жүзінде (практикалық мысал)

Бір ағаш пен кездейсоқ орманның бөлу беттерін бірдей деректерде салыстырайық.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# Есеп күрделі болуы үшін жоғары шу деңгейі бар деректерді генерациялаймыз
X, y = make_moons(n_samples=500, noise=0.3, random_state=42)

# Деректерді оқыту және тесттік таңдамаларға бөлеміз
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# --- ТҮЗЕТУ: Модельдерді ТЕК оқыту таңдамасында оқытамыз ---
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_train)
# ----------------------------------------------------------------

# --- Бөлу беттерін визуализациялау ---
def plot_boundary(ax, model, X_data, y_data, title):
    x_min, x_max = X_data[:, 0].min() - 0.5, X_data[:, 0].max() + 0.5
    y_min, y_max = X_data[:, 1].min() - 0.5, X_data[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='winter')
    # Модельдердің оларды қалай жіктейтінін көру үшін тек тесттік нүктелерді көрсетеміз
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, s=30, edgecolor='k', cmap='winter')
    ax.set_title(title)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
plot_boundary(axes[0], tree_clf, X, y, "Бір Шешімдер Ағашы")
plot_boundary(axes[1], rf_clf, X, y, "Кездейсоқ Орман (100 ағаш)")
plt.suptitle("Бөлу беттерін салыстыру (тесттік деректерде)", fontsize=16)
plt.show()


# --- Модельдерді тесттік таңдамада бағалау ---
y_pred_tree = tree_clf.predict(X_test)
y_pred_rf = rf_clf.predict(X_test)

print("="*60)
print("Classification Report: Бір Шешімдер Ағашы")
print(classification_report(y_test, y_pred_tree))
print("="*60)

print("\n" + "="*60)
print("Classification Report: Кездейсоқ Орман")
print(classification_report(y_test, y_pred_rf))
print("="*60)


# --- Қателер матрицалары ---
cm_tree = confusion_matrix(y_test, y_pred_tree)
cm_rf = confusion_matrix(y_test, y_pred_rf)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

disp_tree = ConfusionMatrixDisplay(confusion_matrix=cm_tree)
disp_tree.plot(ax=axes[0], cmap='Blues', values_format='d')
axes[0].set_title("Қателер матрицасы - Ағаш")

disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf)
disp_rf.plot(ax=axes[1], cmap='Blues', values_format='d')
axes[1].set_title("Қателер матрицасы - Кездейсоқ орман")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Жоғары шу деңгейі бар деректерді генерациялау
X, y = make_moons(n_samples=500, noise=0.3, random_state=42)

# Деректерді оқыту және тесттік таңдамаларға бөлу
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 1. "Жеке тұрған" Шешімдер Ағашын оқыту
# Ол артық оқытылуы үшін тереңдігін шектемейміз
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

# 2. Кездейсоқ Орманды оқыту
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_train)


# --- АҒАШТАРДЫ ВИЗУАЛИЗАЦИЯЛАУ ---

# 3. "Жеке тұрған" Шешімдер Ағашын салу
# График оқылатын болуы үшін көрсету тереңдігін шектейміз (max_depth=3),
# бірақ шын мәнінде ағаштың әлдеқайда терең екенін ұмытпаймыз.
plt.figure(figsize=(20, 10))
plot_tree(tree_clf, 
          max_depth=3, # Тек визуализация үшін шектеу
          feature_names=['feature_0', 'feature_1'], 
          class_names=['Класс 0', 'Класс 1'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("'Жеке тұрған' Шешімдер Ағашының жоғарғы деңгейлері", fontsize=16)
plt.show()


# 4. Орманнан бір ағашты "алып шығып", оны салу
# rf_clf.estimators_ - бұл ормандағы барлық ағаштардың тізімі. Біріншісін алайық.
first_tree_from_forest = rf_clf.estimators_[0]

plt.figure(figsize=(20, 10))
plot_tree(first_tree_from_forest, 
          max_depth=3, # Тек визуализация үшін шектеу
          feature_names=['feature_0', 'feature_1'],
          class_names=['Класс 0', 'Класс 1'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Кездейсоқ Орманнан АЛЫНҒАН бір ағаштың жоғарғы деңгейлері", fontsize=16)
plt.show()

**Қорытынды:** Бір ағаштың шекарасы — "жыртылған" және күрделі, ол әрбір нүктеге мінсіз бейімделуге тырысады. Кездейсоқ Орманның шекарасы — әлдеқайда тегіс және жалпыланған. Ол шуды емес, деректердегі жалпы тенденцияны көрсетеді. Кездейсоқ орманнан алынған бір ағаштың құрылымы, бутстрэпті қолдану мен әрбір түйінде белгілерді кездейсоқ таңдау себебінен, барлық деректерде оқытылған ағаштың құрылымынан айтарлықтай ерекшеленеді. Дәл осы ағаштар арасындағы мәжбүрлі "пікірлердің әртүрлілігі" ансамбльге жоғары дәлдікке және артық оқытылуға төзімділікке қол жеткізуге мүмкіндік береді.

### 7-бөлім: Кездейсоқ Орманның Артықшылықтары мен Кемшіліктері

#### Артықшылықтары:
*   **Болжамдардың жоғары дәлдігі:** Көптеген есептерде сызықтық алгоритмдерден жақсы жұмыс істейді; дәлдігі бустингтің дәлдігімен салыстырмалы.
*   **Шығарындылар мен шуға төзімділік:** Бутстрэп әдісімен таңдамаларды кездейсоқ сэмплингтеу арқылы деректердегі шығарындыларға іс жүзінде сезімтал емес.
*   **Масштабтауға** және белгі мәндерінің басқа монотонды түрлендірулеріне **сезімтал еместігі**.
*   **«Қораптан шыққандай» жұмыс істейді:** Жақсы нәтиже алу үшін параметрлерді мұқият баптауды қажет етпейді.
*   **Көптеген белгілермен және кластармен тиімді жұмыс істейді.**
*   **Сирек артық оқытылады:** Тәжірибеде ағаштарды қосу композицияны әрқашан дерлік жақсартады.
*   **Жоғалған деректермен жақсы жұмыс істейді.**
*   **Параллельдеу** және масштабтау оңай.

#### Кемшіліктері:
*   **Интерпретациялаудың күрделілігі:** Бір ағаштан айырмашылығы, кездейсоқ орманның нәтижелерін интерпретациялау қиынырақ.
*   **Сирек деректермен проблемалар:** Таңдамада өте көп сирек белгілер болғанда (мысалы, мәтіндерді талдау есептерінде) көптеген сызықтық әдістерден нашар жұмыс істейді.
*   **Экстраполяция жасай алмайды:** Жалғыз ағаштар сияқты, орман оқыту таңдамасында көрген диапазонынан тыс мәндерді болжай алмайды.
*   **Категориялық белгілерге қатысты бейімділік:** Көптеген деңгейлері (категориялары) бар категориялық белгілерге артықшылық беруге бейім.
*   **Модельдің үлкен өлшемі:** Сақтау үшін O(N·K) жады қажет, мұндағы N - ағаштар саны, ал K - олардағы түйіндер саны.